#Перевірка статиcтичних гіпотез. Z-test.



0. Зчитайте дані з `data.csv` у змінну data, яка має тип pandas.DataFrame. Ми будемо далі працювати з цією змінною.

In [3]:
import pandas as pd

from google.colab import files
uploaded = files.upload()

Saving data.csv to data.csv


In [6]:
data = pd.read_csv('data.csv', index_col=0)
data.head()

,data
0,17.499453
1,19.662399
2,7.182823
3,29.841625
4,9.239386


Запустіть код нижче. Для коректної робити всіх подальших методів дані мають бути у вигляді одновимірного `numpy.ndarray` вектора та мати розмірність (100,). Така розмірність означає, що дані - одновимірні (колонка), якщо б розмірність була (100, 1), то дані сприймаються методами як двовимірні (таблиця), хоча для нас виглядати можуть так само.

In [7]:
data = data.data.values
data.shape

(100,)

**Завдання 1**. (7 балів)
Зчитані дані - це сума покупок в доларах наших покупців на сайті протягом липня 2025 року.

До цього місяця, ми мали продажі в середньому на чек $20.

Необхідно зʼясувати, чи вийшло у нас статистично значущо **підвищити** середній чек за липень 2025?

Аби дати відповідь - ми проведемо z-test на рівні значущості $\alpha=0.05$ двома способами. В цьому завданні ми будемо виконувати обчислення "вручну" з використанням бібліотек numpy та scipy.stats подібно до прикладу в лекції. Для цього виконайте наступні 7 кроків. Правильне виконання кожного з кроків оцінюється в 1 бал.

1. Запишіть параметри задачі у змінні Python:
    - `sample_mean` - $\bar{x}$, середнє значення в вибірці
    - `population_mean` - $\mu_0$, середнє значення в популяції (тобто те, з яким ми порівнюємо середнє вибіркове значення)
    - `population_std` - $\sigma$, вибіркове стандартне відхилення, яке ми вважаємо, що є рівним ст. відх. популяції, адже маємо достатньо велику вибірку.
    - `sample_size` - $n$, розмір вибірки
    - `alpha` - $\alpha$ рівень значущості
    
    **Увага!** Для обчислення стандартного відхилення маємо скористатись функцією `np.std(your_dataframe, ddof=1)`. Чому так - розбираємо в лекції про t-test.


2. Визначте для цієї задачі:
    - якою є гіпотеза $H_0$
    - якою є альтернативна гіпотеза $H_a$
    - з яким типом тесту ми маємо справу - лівосторонній, правосторонній чи двосторонній.

3. Обчисліть стандартну помилку SE.
4. Розрахуйте z-статистику.
5. Знайдіть критичне z-значення з допомогою бібліотеки stats.
6. Обчисліть p-value з допомогою бібліотеки stats.
7. Прийміть рішення, чи відхиляєте ви гіпотезу $H_0$. Для прийняття рішення зробіть і порівняння z-статистики з критичним значенням, і проаналізуйте p-value.

In [8]:
import numpy as np
from scipy import stats

In [10]:
sample_mean = np.mean(data)
population_mean = 20
population_std = np.std(data, ddof=1)
sample_size = len(data)
alpha = 0.05

In [13]:
# H0: μ = 20
# Ha: μ > 20
# нас цікавить питання чи зріс середній чек
# отже фокусуємося лише на збільшенні, тому
# тест буде правостороннім

In [16]:
SE = population_std / np.sqrt(sample_size)
SE

np.float64(0.625356123240434)

In [18]:
z_stat = (sample_mean - population_mean) / SE
z_stat

np.float64(-0.995347738933505)

In [19]:
z_critical = stats.norm.ppf(1 - alpha)
z_critical

np.float64(1.6448536269514722)

In [20]:
p_value = 1 - stats.norm.cdf(z_stat)
p_value

np.float64(0.8402164165464012)

In [21]:
z_stat > z_critical

np.False_

In [22]:
p_value < alpha

np.False_

In [23]:
# на рівні значущості α = 0.05 немає достатніх статистичних підстав
# для відхилення нульової гіпотези про те, що середній чек дорівнює $20,
# оскільки результат не є "занадто дивним" та "занадто рідкісним".

**Завдання 2.** (3 бали)
Виконайте обчислення z-test з використанням бібліотеки statsmodels.

Отримайте z-статистику та р-значення.

Виведіть p-значення та зробіть висновок, чи ми маємо достатньо доказів, аби стверджувати, що середній чек зріс.

Чи зійшлись значення z-статистику та р-значення в цьому завданні з попередніми обчисленнями?

In [24]:
import numpy as np
from statsmodels.stats.weightstats import ztest

In [25]:
z_stat, p_value_two_sided = ztest(data, value=20)

In [26]:
p_value_one_sided = p_value_two_sided / 2

In [27]:
print(f"z-статистика: {z_stat:.4f}")
print(f"p-value (one-sided): {p_value_one_sided:.4f}")

z-статистика: -0.9953
p-value (one-sided): 0.1598


In [28]:
# z < 0 означає, що вибіркове середнє менше, ніж 20
# p-value ≥ 0.05, отже результат статистично не значущий
# отже у нас немає підстав відхиляти нульову гіпотезу

In [30]:
# загалом значення майже зійшлись: знак z той самий, порядок величин той самий, висновки ідентичні
# мінімальні відмінності (в обчисленні p-value) могли виникнути через округлення σ,
# спосіб оцінки стандартного відхилення, кількість знаків після коми.